In [156]:
import torch
import torch.nn.functional as F

In [157]:
names = open('names.txt', 'r').read().splitlines()

In [158]:
chars = sorted(set('.'.join(names)))

itos = {i:c for i, c in enumerate(chars)}
stoi = {c: i for i, c in enumerate(chars)}

In [290]:
def build_dataset(names):
    block_size = 8
    X, Y = [], []

    for name in names:
        context = [0] * block_size
        for ch in name + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y 

In [291]:
import random

random.seed(42)
random.shuffle(names)
n1 = int(0.8 * len(names))
n2 = int(0.1 * len(names))

Xtr, Ytr = build_dataset(names[:n1])
Xdev, Ydev = build_dataset(names[n1:n2])
Xte, Yte = build_dataset(names[n2:])

In [297]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 25), generator=g)
b1 = torch.randn(200, generator=g)
W1 = torch.randn(200, 200, generator=g)
W2 = torch.randn(200, 27, generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [298]:
lr = torch.linspace(-3, 0, 50000)
lr = reversed(10**lr)

In [299]:
for p in parameters:
    p.requires_grad = True

In [320]:
for i in range(50000):
    ix = torch.randint(0, Xtr.shape[0], (32,))
    emb = C[Xtr[ix]]
    h = torch.tanh(emb.view(-1, 200) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr[ix])
    loss.backward()
    for p in parameters:
        p.data -= 0.01 * p.grad
        p.grad = None



In [311]:
emb = C[Xte]
h = torch.tanh(emb.view(-1, 200) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Yte)
loss

tensor(2.2176, grad_fn=<NllLossBackward0>)

In [319]:
res = []
context_len = 8
for _ in range(10):
    context = [0] * context_len
    name = ''
    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1, 200) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1).item()
        context = context[1:] + [ix]
        name += itos[ix]
        if ix == 0:
            res.append(name)
            break

res


['essihe.',
 'bryertn.',
 'harzoah.',
 'ehla.',
 'zarbree.',
 'daylinut.',
 'daniah.',
 'jockeanca.',
 'vadah.',
 'spav.']